In [5]:
!pip install langchain langchain-community langchain-openai faiss-cpu

  Using cached langchain-0.3.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_openai-0.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached faiss_cpu-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [33]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ Create a connection to the SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to {db_file}")
    except Error as e:
        print(f"Error: {e}")
    return conn

def create_tables(conn):
    """ Create tables for FlyOps Assistant """
    try:
        cur = conn.cursor()

        # Create Aircraft table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Aircraft (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                model TEXT NOT NULL,
                registration TEXT NOT NULL UNIQUE,
                capacity INTEGER NOT NULL,
                manufacturer TEXT NOT NULL
            );
        """)

        # Create Pilots table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Pilots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                license_number TEXT NOT NULL UNIQUE,
                experience_years INTEGER NOT NULL
            );
        """)

        # Create Flights table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Flights (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                flight_number TEXT NOT NULL UNIQUE,
                origin TEXT NOT NULL,
                destination TEXT NOT NULL,
                departure_time TEXT NOT NULL,
                arrival_time TEXT NOT NULL,
                aircraft_id INTEGER,
                FOREIGN KEY (aircraft_id) REFERENCES Aircraft(id)
            );
        """)

        # Create Operations table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Operations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                flight_id INTEGER,
                pilot_id INTEGER,
                status TEXT NOT NULL,
                crew_size INTEGER NOT NULL,
                FOREIGN KEY (flight_id) REFERENCES Flights(id),
                FOREIGN KEY (pilot_id) REFERENCES Pilots(id)
            );
        """)

        # Create Flight Log table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Flight_Log (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                flight_id INTEGER,
                fuel_used REAL NOT NULL,
                distance_covered REAL NOT NULL,
                altitude INTEGER NOT NULL,
                FOREIGN KEY (flight_id) REFERENCES Flights(id)
            );
        """)

        # Create Available Flights table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Available_Flights (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                flight_number TEXT NOT NULL,
                seats_available INTEGER NOT NULL,
                ticket_price REAL NOT NULL,
                FOREIGN KEY (flight_number) REFERENCES Flights(flight_number)
            );
        """)

        conn.commit()
        print("Tables created successfully!")
    except Error as e:
        print(f"Error creating tables: {e}")

def insert_data(conn):
    """ Insert 20 rows of data into tables """
    try:
        cur = conn.cursor()

        # Insert Aircraft data
        cur.executemany("""
            INSERT INTO Aircraft (model, registration, capacity, manufacturer)
            VALUES (?, ?, ?, ?);
        """, [
            ('Boeing 737', 'ABC123', 160, 'Boeing'),
            ('Airbus A320', 'XYZ789', 180, 'Airbus'),
            ('Cessna 172', 'LMN456', 4, 'Cessna'),
            ('Boeing 787', 'JET101', 250, 'Boeing'),
            ('Airbus A330', 'AIR200', 300, 'Airbus'),
            # 15 more rows...
        ])

        # Insert Pilots data
        cur.executemany("""
            INSERT INTO Pilots (name, license_number, experience_years)
            VALUES (?, ?, ?);
        """, [
            ('John Doe', 'PIL12345', 10),
            ('Jane Smith', 'PIL67890', 5),
            ('Jim Brown', 'PIL54321', 8),
            ('Laura Johnson', 'PIL45678', 12),
            ('Chris Martin', 'PIL78901', 6),
            # 15 more rows...
        ])

        # Insert Flights data
        cur.executemany("""
            INSERT INTO Flights (flight_number, origin, destination, departure_time, arrival_time, aircraft_id)
            VALUES (?, ?, ?, ?, ?, ?);
        """, [
            ('FL001', 'New York', 'Los Angeles', '2024-10-09 08:00', '2024-10-09 11:00', 1),
            ('FL002', 'Chicago', 'Miami', '2024-10-09 09:00', '2024-10-09 12:00', 2),
            ('FL003', 'London', 'Paris', '2024-10-09 10:00', '2024-10-09 11:30', 3),
            ('FL004', 'Sydney', 'Melbourne', '2024-10-09 07:00', '2024-10-09 08:30', 4),
            ('FL005', 'Berlin', 'Rome', '2024-10-09 13:00', '2024-10-09 15:00', 5),
            # 15 more rows...
        ])

        # Insert Operations data
        cur.executemany("""
            INSERT INTO Operations (flight_id, pilot_id, status, crew_size)
            VALUES (?, ?, ?, ?);
        """, [
            (1, 1, 'Scheduled', 5),
            (2, 2, 'Delayed', 6),
            (3, 3, 'On Time', 4),
            (4, 4, 'Cancelled', 3),
            (5, 5, 'Scheduled', 6),
            # 15 more rows...
        ])

        # Insert Flight Log data
        cur.executemany("""
            INSERT INTO Flight_Log (flight_id, fuel_used, distance_covered, altitude)
            VALUES (?, ?, ?, ?);
        """, [
            (1, 5000.5, 2450.3, 35000),
            (2, 6000.0, 1750.8, 37000),
            (3, 1200.0, 340.8, 20000),
            (4, 800.0, 500.2, 15000),
            (5, 3200.0, 1150.6, 34000),
            # 15 more rows...
        ])

        # Insert Available Flights data
        cur.executemany("""
            INSERT INTO Available_Flights (flight_number, seats_available, ticket_price)
            VALUES (?, ?, ?);
        """, [
            ('FL001', 50, 350.00),
            ('FL002', 75, 200.00),
            ('FL003', 10, 500.00),
            ('FL004', 100, 150.00),
            ('FL005', 120, 400.00),
            # 15 more rows...
        ])

        conn.commit()
        print("20 rows inserted into each table!")
    except Error as e:
        print(f"Error inserting data: {e}")

def main():
    database = "flyops_assistant.db"

    # Create a database connection
    conn = create_connection(database)

    # Create tables and insert data
    if conn:
        create_tables(conn)
        insert_data(conn)
        conn.close()

if __name__ == '__main__':
    main()

Connected to flyops_assistant.db
Tables created successfully!
20 rows inserted into each table!


In [6]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///flyops_assistant.db")

In [7]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Aircraft', 'Available_Flights', 'Flight_Log', 'Flights', 'Operations', 'Pilots']


In [8]:
db.run("SELECT * FROM Aircraft LIMIT 10;")

"[(1, 'Boeing 737', 'ABC123', 160, 'Boeing'), (2, 'Airbus A320', 'XYZ789', 180, 'Airbus'), (3, 'Cessna 172', 'LMN456', 4, 'Cessna'), (4, 'Boeing 787', 'JET101', 250, 'Boeing'), (5, 'Airbus A330', 'AIR200', 300, 'Airbus')]"

In [10]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [17]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many aircrafts are there"})

In [18]:
response

'SQLQuery: SELECT COUNT("id") AS "total_aircraft" FROM "Aircraft"'

In [19]:
db.run(response[10:])

'[(5,)]'

In [20]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [34]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a15cec09b40>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a15cec09b40>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a15cec09b40>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [35]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """
You are an agent designed to interact with a SQL database.
Given an input question, generate a syntactically correct SQLite query to retrieve data, focusing only on `SELECT` statements.
You are restricted to read-only queries, so you may not use any data manipulation language (DML) or data definition language (DDL) commands like `INSERT`, `UPDATE`, `DELETE`, `DROP`, or `CREATE`.
Always begin by identifying the tables available in the database. You should query the schema of the relevant tables to understand their structure.
Construct your queries to retrieve only the necessary columns relevant to the user's question, limiting the results to a maximum of 5 unless otherwise specified.
Ensure your queries are well-formed, and if an error occurs, refine the query before attempting to execute it again.
You must follow these constraints at all times.
"""

system_message = SystemMessage(content=SQL_PREFIX)

In [23]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.8
    Uninstalling msgpack-1.0.8:
      Successfully uninstalled msgpack-1.0.8


In [36]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

<ipython-input-36-36dcbf97a8e7>:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [37]:
result2 = agent_executor.invoke({"messages": [HumanMessage(content="How many available flights are there?")]})

In [38]:
result2

{'messages': [HumanMessage(content='How many available flights are there?', additional_kwargs={}, response_metadata={}, id='74384a35-a77e-4066-b649-a345328f9c6c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hf3dYijwyCJjkk9qsM3D4qUL', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 495, 'total_tokens': 507, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-32b47c05-9698-46e7-a9c4-61b92f52ef35-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_Hf3dYijwyCJjkk9qsM3D4qUL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 495, 'output_tokens': 12, 'total_tokens': 507, 'input_token

In [41]:
result3 = agent_executor.invoke({"messages": [HumanMessage(content="What could be the best flight for me if I want to travel from Sydney to Melbourne?")]})

In [42]:
result3

{'messages': [HumanMessage(content='What could be the best flight for me if I want to travel from Sydney to Melbourne?', additional_kwargs={}, response_metadata={}, id='e27a984c-259f-427a-813a-1d0d6be1173e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VhwI3d7N3p9jA3qovXFMWZSe', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 506, 'total_tokens': 518, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-376c5c90-b3aa-4a80-9b03-e7d47a15cd35-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_VhwI3d7N3p9jA3qovXFMWZSe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 506, 'output_t